# Non-tagged Productivity in All Namespaces

This notebook is similar to 12-nontagged-article-productivity.ipynb, except it uses edits across all namespaces and excludes non-tagged edits.

It makes the same cutoff for registration timestamp as 12-nontagged-article-productivity.ipynb, for the same reasons.

In [1]:
# https://stackoverflow.com/a/35018739/1091835
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [24]:
library(data.table)
library(ggplot2)

library(brms) # install.packages("brms")
library(loo) # install.packages("loo")
options(mc.cores = 4)
library(rstanarm) # install.packages("rstanarm")

library(lme4)

Loading required package: Matrix


Attaching package: ‘lme4’


The following object is masked from ‘package:brms’:

    ngrps




## Configuration variables

In [3]:
## Set BLAS threads to 4 so glmer and loo don't use all cores
library(RhpcBLASctl)
blas_set_num_threads(1)

## parallelization
options(mc.cores = 4)

### Data import and setup

In [4]:
nontagged_edit_data = fread(
    '/home/nettrom/src/Growth-homepage-2019/datasets/newcomer_tasks_nontagged_edits_nov2020.tsv',
    colClasses = c(wiki_db = 'factor'))

In [26]:
## Configuration variables for this experiment.
## Start timestamp is from https://phabricator.wikimedia.org/T227728#5680453
## End timestamp is from the data gathering notebook
start_ts = as.POSIXct('2019-11-21 00:24:32', tz = 'UTC')
end_ts = as.POSIXct('2020-04-9 00:00', tz = 'UTC')

## Start of the Variant A/B test
variant_test_ts = as.POSIXct('2019-12-13 00:32:04', tz = 'UTC')

## Convert user_registration into a timestamp
nontagged_edit_data[, user_reg_ts := as.POSIXct(user_registration_timestamp,
                                           format = '%Y-%m-%d %H:%M:%S.0', tz = 'UTC')]

## Calculate time since start of experiment in weeks
nontagged_edit_data[, exp_days := 0]
nontagged_edit_data[, exp_days := difftime(user_reg_ts, start_ts, units = 'days')]
nontagged_edit_data[exp_days < 0, exp_days := 0]
nontagged_edit_data[, ln_exp_days := log(1 + as.numeric(exp_days))]
nontagged_edit_data[, ln_exp_weeks := log(1 + as.numeric(exp_days)/7)]

## Calculate time since the start of the variant test, again in days and weeks.
## This enables us to do an interrupted time-series model for that.
nontagged_edit_data[, variant_exp_days := 0]
nontagged_edit_data[, variant_exp_days := difftime(user_reg_ts, variant_test_ts, units = 'days')]
nontagged_edit_data[variant_exp_days < 0, variant_exp_days := 0]
nontagged_edit_data[, ln_var_exp_days := log(1 + as.numeric(variant_exp_days))]
nontagged_edit_data[, ln_var_exp_weeks := log(1 + as.numeric(variant_exp_days)/7)]
nontagged_edit_data[, in_var_exp := 0]
nontagged_edit_data[user_reg_ts > variant_test_ts, in_var_exp := 1]

## Convert all NAs to 0, from
## https://stackoverflow.com/questions/7235657/fastest-way-to-replace-nas-in-a-large-data-table
na_to_zero = function(DT) {
  # or by number (slightly faster than by name) :
  for (j in seq_len(ncol(DT)))
    set(DT,which(is.na(DT[[j]])),j,0)
}

na_to_zero(nontagged_edit_data)

## Turn "reg_on_mobile" into a factor for more meaningful plots
nontagged_edit_data[, platform := 'desktop']
nontagged_edit_data[reg_on_mobile == 1, platform := 'mobile']
nontagged_edit_data[, platform := factor(platform)]

## Control variables for various forms of activation
nontagged_edit_data[, is_activated_article := num_article_edits_24hrs > 0]
nontagged_edit_data[, is_activated_other := num_other_edits_24hrs > 0]
nontagged_edit_data[, is_activated := is_activated_article | is_activated_other]

## Control variables for constructive forms of activation
nontagged_edit_data[, is_const_activated_article := (num_article_edits_24hrs - num_article_reverts_24hrs) > 0]
nontagged_edit_data[, is_const_activated_other := (num_other_edits_24hrs - num_other_reverts_24hrs) > 0]
nontagged_edit_data[, is_const_activated := is_const_activated_article | is_const_activated_other]

## Control variables for the number of edits made
nontagged_edit_data[, log_num_article_edits_24hrs := log(1 + num_article_edits_24hrs)]
nontagged_edit_data[, log_num_other_edits_24hrs := log(1 + num_other_edits_24hrs)]
nontagged_edit_data[, log_num_edits_24hrs := log(1 + num_article_edits_24hrs + num_other_edits_24hrs)]

## Control variables for the constructive number of edits made
nontagged_edit_data[, log_num_const_article_edits_24hrs := log(
    1 + num_article_edits_24hrs - num_article_reverts_24hrs)]
nontagged_edit_data[, log_num_const_other_edits_24hrs := log(
    1 + num_other_edits_24hrs - num_other_reverts_24hrs)]
nontagged_edit_data[, log_num_const_edits_24hrs := log(
    1 + num_article_edits_24hrs + num_other_edits_24hrs -
    num_article_reverts_24hrs - num_other_reverts_24hrs)]

## Retention variables
nontagged_edit_data[, is_const_retained_article := is_activated_article &
               ((num_article_edits_2w - num_article_reverts_2w) > 0)]
nontagged_edit_data[, is_const_retained_other := is_const_activated_other &
               ((num_other_edits_2w - num_other_reverts_2w) > 0)]
nontagged_edit_data[, is_const_retained := is_const_activated &
               ((num_article_edits_2w + num_other_edits_2w -
                 num_article_reverts_2w - num_other_reverts_2w) > 0)]

## Variables for number of edits (overall and only constructive)
## across the entire period.
nontagged_edit_data[, num_total_edits_24hrs := num_article_edits_24hrs + num_other_edits_24hrs]
nontagged_edit_data[, num_total_edits_2w := num_article_edits_2w + num_other_edits_2w]
nontagged_edit_data[, num_total_edits := num_total_edits_24hrs + num_total_edits_2w]

nontagged_edit_data[, num_total_const_edits_24hrs := (num_article_edits_24hrs + num_other_edits_24hrs -
                                                 num_article_reverts_24hrs - num_other_reverts_24hrs)]
nontagged_edit_data[, num_total_const_edits_2w := (num_article_edits_2w + num_other_edits_2w -
                                              num_article_reverts_2w - num_other_reverts_2w)]
nontagged_edit_data[, num_total_const_edits := num_total_const_edits_24hrs + num_total_const_edits_2w]

## Variables for number of article edits across the entire period.
nontagged_edit_data[, num_total_article_edits := num_article_edits_24hrs + num_article_edits_2w]

In [28]:
## Registration cutoff (see notes above)
reg_cutoff = as.POSIXct('2019-12-13 09:00:00', tz = 'UTC')

eligible_user_edit_data = nontagged_edit_data[user_reg_ts > reg_cutoff]

## Priors

In [7]:
## Note that using a student_t distribution for the prior is beneficial because that
## distribution handles outliers better than a Normal.
## See https://jrnold.github.io/bayesian_notes/robust-regression.html
## Thanks to Mikhail for sending that to me!
priors <- prior(cauchy(0, 2), class = sd) +
  prior(student_t(5, 0, 10), class = b)

## Edits across the entire 15-day period

We base this model on the same one used across all namespaces, meaning that we don't expect group-level variation in the effect of mobile. This is mainly because we have few wikis in our dataset, thus we don't expect that to contain meaningful information.

In [ ]:
nontagged_edits_all_days_all_namespaces.zinb.mod.1 <- brm(
  bf(num_total_edits ~ is_treatment + reg_on_mobile + (1 | wiki_db),
     zi ~ wiki_db + reg_on_mobile),
    data = eligible_user_edit_data,
    family = zero_inflated_negbinomial(),
    prior = priors,
    iter = 800,
    control = list(adapt_delta = 0.999,
                 max_treedepth = 15)
)

Compiling Stan program...

Start sampling



In [ ]:
## Save the model
save(nontagged_edits_all_days_all_namespaces.zinb.mod.1,
     file='models/nontagged_edits_all_days_all_namespaces.zinb.mod.1.Robj')

In [13]:
summary(nontagged_edits_all_days_all_namespaces.zinb.mod.1)

 Family: zero_inflated_negbinomial 
  Links: mu = log; shape = identity; zi = logit 
Formula: num_total_edits ~ is_treatment + reg_on_mobile + (1 | wiki_db) 
         zi ~ wiki_db + reg_on_mobile
   Data: eligible_user_edit_data (Number of observations: 85235) 
Samples: 4 chains, each with iter = 800; warmup = 400; thin = 1;
         total post-warmup samples = 1600

Group-Level Effects: 
~wiki_db (Number of levels: 4) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     1.08      0.70     0.42     3.04 1.00      490      588

Population-Level Effects: 
                 Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept            0.42      0.62    -1.04     1.47 1.01      306      363
zi_Intercept        -8.81      1.95   -13.61    -6.17 1.01      555      446
is_treatment        -0.08      0.02    -0.13    -0.04 1.00     1581     1099
reg_on_mobile        0.34      0.02     0.30     0.38 1.00     1536     1322
zi_wiki_dbcswik

In [ ]:
pp_check(nontagged_article_edits_all_days.zinb.mod.1, 
         nsamples = 100, type = "bars_grouped", group = "wiki_db") +
  ggplot2::coord_cartesian(xlim = c(0, 10))

Zooming in on the non-zero part of the distributions, because the high zero-rate results in such a compressed graph.

In [ ]:
pp_check(nontagged_article_edits_all_days.zinb.mod.1, 
         nsamples = 100, type = "bars_grouped", group = "wiki_db") +
  ggplot2::coord_cartesian(xlim = c(1, 10), ylim = c(0, 10000))

This pattern is similar to that of the overall model in that it underestimates the number of ones, and sometimes twos. Something to keep in mind moving forward, if there are ways for us to improve on that.

In [ ]:
## Get predictions from the model about what the expected number of edits
## are for a user in the Control and Homepage groups for all wikis.
predict_data = data.frame(
    wiki_db = c(rep('arwiki', 4), rep('cswiki', 4),
                rep('kowiki', 4), rep('viwiki', 4)),
    reg_on_mobile = rep(c(0, 0, 1, 1), 4),
    is_treatment = rep(c(0, 1), 8),
    in_var_exp = rep(0, 16))

cbind(predict_data,
      round(predict(nontagged_article_edits_all_days.zinb.mod.1, predict_data, type = "response"), 1))

## Estimated effects

First the average number of edits in the control group:

In [11]:
ctrl_grp_mean = exp(mean(log(1 + eligible_user_edit_data[is_treatment == 0]$num_total_edits))) -1
ctrl_grp_mean

[1] 0.4577414

The average number of edits in the Homepage group:

In [14]:
homepage_grp_mean = exp(fixef(nontagged_edits_all_days_all_namespaces.zinb.mod.1, pars = 'is_treatment')[1] +
                        mean(log(1 + eligible_user_edit_data[is_treatment == 0]$num_total_edits))) -1
homepage_grp_mean

[1] 0.3413626

In [15]:
(ctrl_grp_mean - homepage_grp_mean)

[1] 0.1163787

Let's find the 95% credible interval:

In [16]:
homepage_grp_low = exp(fixef(nontagged_edits_all_days_all_namespaces.zinb.mod.1, pars = 'is_treatment')[3] +
                       mean(log(1 + eligible_user_edit_data[is_treatment == 0]$num_total_edits))) -1
homepage_grp_low

[1] 0.284353

In [17]:
(ctrl_grp_mean - homepage_grp_low)

[1] 0.1733883

In [18]:
homepage_grp_high = exp(fixef(nontagged_edits_all_days_all_namespaces.zinb.mod.1, pars = 'is_treatment')[4] +
                        mean(log(1 + eligible_user_edit_data[is_treatment == 0]$num_total_edits))) -1
homepage_grp_high

[1] 0.3994667

In [19]:
(ctrl_grp_mean - homepage_grp_high)

[1] 0.05827468

In [20]:
(ctrl_grp_mean - homepage_grp_mean) / ctrl_grp_mean

[1] 0.2542456

In [21]:
(ctrl_grp_mean - homepage_grp_low) / ctrl_grp_mean

[1] 0.3787911

In [22]:
(ctrl_grp_mean - homepage_grp_high) / ctrl_grp_mean

[1] 0.1273092

In summary, we find that the Control group makes an average of $0.46$ edits across all namespaces during the activation and retention periods (compare this to $0.5$ edits reported in the Newcomer Tasks Experiment Analysis report). The Homepage group is estimated to make an average of $0.34$ non-tagged edits ($-0.12$ edits or $-25.4\%$). We're 95% confident the Homepage group's estimate is in the interval $[0.28, 0.40]$ edits, which is in the interval $[-0.17, -0.06]$ edits relative to the Control group, or $[-37.9\%, -12.7\%]$